In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import sklearn as sk
import seaborn as sns

In [166]:
REWARD_NONTERMINAL = -1
REWARD_TERMINAL = 10
REWARD_CLIFF = -100

class Environment:

    def __init__(self, nr_columns, nr_rows, nr_actions=4, init_qa_values=0):
        self.world = np.zeros((nr_rows + 1, nr_columns))
        self.nr_columns = nr_columns
        self.nr_rows = nr_rows
        self.nr_actions = nr_actions
        
    def setWorldRewards(self):
        pass

class CliffEnvironment(Environment):
    
    def __init__(self, nr_columns, nr_rows, nr_actions=4, init_qa_values=0):
        super().__init__(nr_columns, nr_rows)
        self.setWorldRewards()
        
    def setWorldRewards(self):
        self.world[:, :] = REWARD_NONTERMINAL
        self.world[self.nr_rows:, 1:self.nr_columns - 1] = REWARD_CLIFF
        self.world[self.nr_rows, self.nr_columns - 1] = REWARD_TERMINAL

In [168]:
class Agent:
    def __init__(self, env, init_position=(0,0)):
        self.position = init_position
        self.env = env
        self.q_table = np.zeros((env.nr_columns, env.nr_rows + 1, env.nr_actions))
        
    def run(self):
        pass
        
class QLearner(Agent):
    def __init__(self, env, init_position=(0,0)):
        super().__init__(env, init_position)
        
        
    def run(self):
        pass
#       Walk, and get reward for the next state
# 
# 
# 
    
class SarsaLearner(Agent):
    def __init__(self, env, init_position=(0,0)):
        super().__init__(env)
        
    def run(self):
        pass

In [164]:
###
# Main Run - Initialization
###
NR_COLUMNS = 5

# Excluding the cliff
NR_ROWS = 3

cliffWorld = CliffEnvironment(NR_COLUMNS, NR_ROWS)
agent = QLearner(cliffWorld, (0, NR_ROWS))

In [165]:
print(agent.q_table)

[[[0. 0. 0. 0.]
  [0. 0. 0. 0.]
  [0. 0. 0. 0.]
  [0. 0. 0. 0.]]

 [[0. 0. 0. 0.]
  [0. 0. 0. 0.]
  [0. 0. 0. 0.]
  [0. 0. 0. 0.]]

 [[0. 0. 0. 0.]
  [0. 0. 0. 0.]
  [0. 0. 0. 0.]
  [0. 0. 0. 0.]]

 [[0. 0. 0. 0.]
  [0. 0. 0. 0.]
  [0. 0. 0. 0.]
  [0. 0. 0. 0.]]

 [[0. 0. 0. 0.]
  [0. 0. 0. 0.]
  [0. 0. 0. 0.]
  [0. 0. 0. 0.]]]
